In [ ]:
import carla
import numpy as np
import cv2
import time
import joblib  # To load/save sklearn models
import matplotlib.pyplot as plt
import pygame
from carla import VehicleLightState

# === Pygame setup ===
pygame.init()
width, height = 800, 600
screen = pygame.display.set_mode((width, height))
pygame.display.set_caption("CARLA Manual Control")

# --- Connect to CARLA ---
client = carla.Client('192.168.86.116', 2000)
client.set_timeout(15.0)
client.load_world("Town05")
world = client.get_world()

# --- Get vehicle and attach camera sensor ---
blueprint_library = world.get_blueprint_library()
vehicle_bp = blueprint_library.find('vehicle.tesla.model3')
spawn_points = world.get_map().get_spawn_points()
for spawn_point in spawn_points:
    vehicle = world.try_spawn_actor(vehicle_bp, spawn_point)
    if vehicle is not None:
        break

if vehicle is None:
    raise RuntimeError("Could not spawn vehicle due to collisions.")
    

# Set up camera sensor
camera_bp = blueprint_library.find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '320')
camera_bp.set_attribute('image_size_y', '240')
camera_bp.set_attribute('fov', '110')

camera_transform = carla.Transform(carla.Location(x=1.5, z=2.4))
camera = world.spawn_actor(camera_bp, camera_transform, attach_to=vehicle)

# Shared variable to store the latest camera frame
camera_image = None

def process_image(image):
    global camera_image
    # Convert raw data to numpy array (uint8)
    array = np.frombuffer(image.raw_data, dtype=np.uint8)
    # Reshape to (height, width, 4)
    array = array.reshape((image.height, image.width, 4))
    # Drop alpha channel and convert BGRA -> RGB by reversing last axis
    array = array[:, :, :3][:, :, ::-1]
    # Convert to surface for Pygame (swap axes to width x height)
    camera_image_raw = pygame.surfarray.make_surface(array.swapaxes(0, 1))
    # Stretching image
    camera_image = pygame.transform.scale(camera_image_raw, (800, 600))
    
# === Clock ===
clock = pygame.time.Clock()

# --- Load Trained Model ---
model = joblib.load("../output/baselineModelOLD.pkl")  

# --- Image preprocessing ---
def preprocess(image_array):
    img = cv2.resize(image_array, (160, 120))  # Resize to match training size
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    img = img / 255.0  # Normalize
    return img.flatten().reshape(1, -1)  # Shape: (1, 4800)

# === Dynamic spectator follow camera ===
spectator = world.get_spectator()
transform = vehicle.get_transform()
forward_vector = transform.get_forward_vector()
cam_location = transform.location - forward_vector * 8 + carla.Location(z=3)
cam_rotation = carla.Rotation(pitch=-10, yaw=transform.rotation.yaw)
spectator.set_transform(carla.Transform(cam_location, cam_rotation))

# --- Control Loop ---
def drive():
    control = carla.VehicleControl()
    
    def on_image(image):
        # Convert CARLA image to numpy array
        img = np.frombuffer(image.raw_data, dtype=np.uint8)
        img = img.reshape((image.height, image.width, 4))[:, :, :3]  # Drop alpha channel

        # Preprocess
        X_input = preprocess(img)

        # Predict steering angle
        steer = model.predict(X_input)[0]
        steer = float(np.clip(steer, -1.0, 1.0))  # Clamp to valid range

        # Drive
        control.steer = steer
        control.throttle = 0.4
        control.brake = 0.0
        vehicle.apply_control(control)

        transform = vehicle.get_transform()
        forward_vector = transform.get_forward_vector()
        cam_location = transform.location - forward_vector * 8 + carla.Location(z=3)
        cam_rotation = carla.Rotation(pitch=-10, yaw=transform.rotation.yaw)
        spectator.set_transform(carla.Transform(cam_location, cam_rotation))

        # === Display camera feed ===
        process_image(image)
        

    # Register callback
    camera.listen(lambda image: on_image(image))

try:
    drive()

    while True:
        if camera_image:
            screen.blit(camera_image, (0, 0))
            pygame.display.flip()
        # Exit
        keys = pygame.key.get_pressed()

        # Turn signals
        left_indicator_light = keys[pygame.K_q]
        right_indicator_light = keys[pygame.K_e]

        if left_indicator_light:
            vehicle.set_light_state(VehicleLightState.LeftBlinker)
            time.sleep(3)
            vehicle.set_light_state(VehicleLightState.NONE)


        if right_indicator_light:
            vehicle.set_light_state(VehicleLightState.RightBlinker)
            time.sleep(3)
            vehicle.set_light_state(VehicleLightState.NONE)



        for event in pygame.event.get():
            if event.type == pygame.QUIT or keys[pygame.K_ESCAPE]:
                raise KeyboardInterrupt


finally:
    camera.stop()
    vehicle.destroy()
    camera.destroy()
    pygame.quit()

